# Other application of ConvNets

### What is Face Recognition?

Face recognition and lifeness detection

Face verification vs Face recognition. 
Verification (1 to 1 problem)

Recognition is much harder. 

### One-shot learning problem 
(Given one example, you need to recognize similar ones)
This is needed for most verification system. 

Feed a new image to ConvNet and pass a soft-max with probabilities for various people.  

Learn a `similarity` function. 
$d(\text{img}_1,\text{img}_2) \leq\tau$ -> predict the "same" else "different" people.  

Function $d$ does pair-wise comparison for each new face/image to compare with those used in training.  

### Siamese Network

In this __ConvNet__ converts image to a FC layer, $f(x^{(1)})$, an __encoding__ of an image.  
The same is done for other images, -- _encoding_ vectors are created for each image.  

Run $N$ identical NNs on N different inputs and compare the results as follows

Define than distance $d(x^{(1)},x^{(2)})$ as a distance between two images $|| f(x^{(1)}) - f(x^{(2)})||$.
If same person $d(...)<\tau$

This is called _siamese network_. (See paper by ) Yaniv Taigman 2014 DeepFace.  

### Triplet Loss Function

Comparing pairs of images. Several pictures at the same time should be analyzed. 

In `triplet loss` function one compares the `anchor` with `positive (P)` image (which is the same, but different) and `negative (N)` which is differnt object/person. 

The following propertiy is desired:
$$
|| f(x^{(A)}) - f(x^{(P)})||^2 \leq 
|| f(x^{(A)}) - f(x^{(N)})||^2
$$

i.e., comparing distances. 

Modification is needed to avoid zeroes in all encodings (trivial solution) 

$$
d(x^{(A)},x^{(P)}) - d(x^{(A)},x^{(N)}) +\alpha \leq 0
$$

where $\alpha$ is `margine`. 

The `triplet loss function` is defined for three images as: 

$$
\mathcal{L} = \max\Big( 
    || f(x^{(A)}) - f(x^{(P)})||^2 - 
    || f(x^{(A)}) - f(x^{(N)})||^2 + \alpha, 0\Big)
$$

So, the expression can be as negative as possible, but less then 0.

$$
j = \sum\mathcal{L}(A^{(i)},P^{(i)},N^{(i)})
$$

Note, pairs of pictures are needed. Large number of pairs of pictures. After training, one picture should be enough. 

**Note** In paris of images are chose randomly, than the constraint $d(A,P) + \alpha \leq d(A,N)$ is easy to satisdy. 

**Triplets have to be chosen so that they are hard to classify**

Tis increases the computational efficiently of the training.  

Schroff et al 2015 FaceNet paper. 

All of this is made to learn an encoding. 

### Face verification and binary classification

Another way to do face regnition.  

This is done by combining these different nets and outputing a signle softmax output $\hat{y}$, computing the __difference between encodings__. 

$$
\hat{y} = \sigma\Big( \sum_{k=1}^{128} | f(x^{(i)})_k - f(x^{(j)})_j | + b \Big)
$$

feeding this into _logistic regresion_ wit a stanrard unit and outputing the probability of a similarity.  

Sometimes $\chi^2$ similarity is also employed. 

Note, that _precomputing_ embeding can save computational cost. 

### Neural Style Transfer 

Convert a style of one omage into a style of a different image. 
Consider $C$ as a content image, $S$ as a style image and $G$ as a generated image.  

THis is done by considering features learned at various layers of a _ConvNet_. 

### What are deep ConvNets learning?

Consider a ConvNet and exampine its layers.  
Take all the training examples, pass through the Net and fine __what image maximises the activation values in this layer__. Plot the image patches that the layer leared.
Usually, these are basic shapes, like lines. 

Repeat the procedure for other hidden units. 
This is called `receptive field`. 

Hidden units in lyaer one are learning simple features. 

See Paper by Zieler et al 2013 Visualazing and understanding ConvNets

Now consider _deeper layers_ where NN sees __Larger__ part of the image, and they are learning features seen with a larger _receprive field_.  

Each layer is detecting more sophysitcated shapes and categories of objects. 

### Cost function for neural Style transfer

First. Define $J(G)$. 
1. Contect part 
2. Style part

$$
J = \alpha J_{content}(C,G) + \beta J_{style}(S,G)
$$

Paper suggested Gatys et al 2015 A neural algorithm of artistic style 

- Init the G randomly (white noise image)
- Define and minimize $J(G)$ and update the $G:=G-\frac{\partial}{\partial G}J(G)$

This will slowly turn the noise-image into the image with the content and style. 

### Content Cost Function

Consider layer $l$, there $a^{[l][C]}$ and $a^{[l][G]}$ activations hwen content and generated image are passed and the cost fanction there then is 
$$
J_{\rm content}(C,G) = \frac{1}{2} || a^{[l][C]} - a^{[l][G]} ||
$$
Not that for this a _pre-trained NN_ (like __VGG__) was used.  

### Style Cost Function

Consider a ConvNet with input in three color channels and a set of lyaers, converting the width and height of the image to the depth.

Define a style as a `correlation between activations across different channels`. 

Channels are related to colors. 
Examining two chanels, one can look at the pairs for a given position $H$ and $W, and how correlated are these numbers across the enitre image. 

See paper by Garys et al 2015 A neural algorithm of artistic style 

Correlation says which of the high level components do or do not occure together. 
Degree of correlation gives how often these features do occure (texture, colors, etc). 

Given an image, compute the _style matrix_ as $a_{ijk}^{[l]} = $activation at $(i,j,k)$ $G^{[l]}$ is $n_c^{[l]}\times n_c^{[l]}$

One element is computed as __unnormalized cross-covaraince__ grand matrixes:

$$
G_{kk'}^{[l][S]} = \sum_{i=1}^{n_{\rm H}}\sum_{j=1}^{n_{\rm W}} a_{ijk}^{[l][S]} a_{ijk'}^{[l][S]}
$$

$$
G_{kk'}^{[l][G]} = \sum_{i=1}^{n_{\rm H}}\sum_{j=1}^{n_{\rm W}} a_{ijk}^{[l][G]} a_{ijk'}^{[l][G]}
$$

These two matrixes capture the styles of image S and G.  

And the style cost function reads 

$$
J_{\rm style}^{[l]}(S,G) = \frac{1}{(2 n_{\rm H}^{[l]} n_{\rm W}^{[l]} n_{\rm C}^{[l]})^2} \sum_k \sum_{k'} \big(  G_{kk'}^{[l][S]} - G_{kk'}^{[l][G]}\big)^2
$$

The overall cost function than is a weighted sum 

$$
J_{\rm style}(S,G) = \sum_l \lambda^{[l]} J_{\r style}^{[l]}(S,G)
$$

with a hyererparameter controls how much various layers learn.  

The overall cost function now is 
$$
J(G) = \alpha J_{\rm cost}(C,G) + \beta J_{\rm style}(S,G)
$$


### Convolutions in 1D and 3D generalization

ConvNets acan be used for 1D or 3D data as well.  

For a signal, for example, EGG diagram, one can use a 1D ConvNet (_time series_ analysis) wih 1D filter. Simialar, the signal time resolutio shrinks each time the convolution is done.   

Mostly, hpwever, __recurrent NNs__ are used.  



